In [63]:
#test scrape
import json
import re
import requests
from bs4 import *
import time
import pandas as pd
from datetime import date

In [21]:
def available(pattern, found):
    if re.search(pattern, str(found)):
        return 1
    else:
        return 0

def isPagination(toggle, url, page):
    if not toggle:
        return url
    if toggle:
        return f'{url[:-1]}{{{page}}}'

def paginationResponse(toggle, url, page):
    if toggle:
        tmp = f'{isPagination(toggle, url, page)}'
        activeUrl = f"f'{tmp}'"
        return requests.get(eval(activeUrl))
    if not toggle:
        return requests.get(url)

In [110]:
def scrape_extract(url:str):
    mdf = pd.DataFrame(columns=['date', 'link', 'price', 'name', 'delivery', 'store'])
    response = requests.get(url)
    html = response.text
    soup = BeautifulSoup(html, 'lxml')
    pattern = '.\d$'
    try:
        lastPage = int(re.search(pattern, soup.find('div', class_='mak-pagination-new').ul.text)[0])
        toggle = True
    except:
        lastPage = 1
        toggle = False

    for i in range(0, lastPage):
        response = paginationResponse(toggle, url, page=i)
        soup = BeautifulSoup(response.text, 'lxml')

        ### product and price
        result = soup.find_all('div', class_='mak-product-tiles-container__product-tile')
        
        for index, j in enumerate(result):
            # extract price and name
            # price = result[index].find('p', class_='col-xs-12').text
            price = j.find('p', class_='col-xs-12').text
            name = result[index].find('a', class_='product-tile-inner__productTitle').span.text
            priceList = ["".join(re.findall('\d', x.find('p', class_='col-xs-12').text)) for x in result]
            nameList = [x.find('a', class_='product-tile-inner__productTitle').span.text for x in result]

            # extract in-store and delivery availability 
            storePattern = 'Store'
            deliveryPattern = 'Delivery'
            availability = [str(x.find_all('span', {'class' : 'fulfillment'})) for x in result]
            storeAvailability = [available(storePattern, x) for x in availability]
            deliveryAvailability = [available(deliveryPattern, x) for x in availability]

            # extract product page link
            link_list = ['www.makro.co.za' + x.find_all('a', class_='product-tile-inner__img', href=True)[0]['href'] for x in result]

            # extract rating
            # rating = [x.find_all('div', class_='bv_text') for x in result]


            df = pd.DataFrame({'date':date.today(), 'link':link_list, 'price':priceList, 'name':nameList, 'delivery':deliveryAvailability, 'store':storeAvailability})        # print(mdf)
        mdf = pd.concat([df, mdf])
    return mdf

In [111]:
# https://www.makro.co.za/food/cooking/grains-rice-pasta/c/ICB?pageSize=20&q=%3Arelevance&page=0
# paginationResponse(True, 'https://www.makro.co.za/books/children-young-adult/young-adult/c/NAC?q=%3Arelevance&page=0', page=1)
mdf = scrape_extract('https://www.makro.co.za/games-gaming/video-games/consoles/c/CAA')
mdf

,date,link,price,name,delivery,store
0,2022-06-22,www.makro.co.za/games-gaming/video-games/conso...,1399900,Xbox Series X 1 TB FIFA 22 Bundle,1,1
1,2022-06-22,www.makro.co.za/games-gaming/video-games/conso...,1399900,PS5 825GB Mortal Kombat 11 Ultimate Bundle,1,1
2,2022-06-22,www.makro.co.za/games-gaming/video-games/conso...,24900,Nitho XB1 Dual Charge and Play Cable,1,0
3,2022-06-22,www.makro.co.za/games-gaming/video-games/conso...,99900,Nitho ADONIS PS4 Wireless Controller,1,0
4,2022-06-22,www.makro.co.za/games-gaming/video-games/conso...,49900,Nitho PS4 Multistand Pro,1,0
5,2022-06-22,www.makro.co.za/games-gaming/video-games/conso...,29900,Nitho XB1 Hyper Battery Pack - 32 Hours,1,0
6,2022-06-22,www.makro.co.za/games-gaming/video-games/conso...,29900,Nitho XB1 Twin Battery Pack,1,0
7,2022-06-22,www.makro.co.za/games-gaming/video-games/conso...,99900,Nitho ADONIS PS4 Wireless Controller CAMO,1,0
8,2022-06-22,www.makro.co.za/games-gaming/video-games/conso...,39900,Nitho PS4 Charging Station,1,0
9,2022-06-22,www.makro.co.za/games-gaming/video-games/conso...,1049500,Oculus Quest 2 — Advanced All-In-One Virtual ...,1,0


In [109]:
# response = requests.get(f"https://www.makro.co.za/games-gaming/video-games/consoles/c/CAA")
# html = response.text
# soup = BeautifulSoup(html, 'lxml')

# ### product and price
# response = soup.find_all("div", class_='rating')
# response[0]
# # # result[0]['href']
# # result

type("".join(re.findall('\d', 'R 12,000000')))



str

In [14]:
pattern = 'Stores'
lastPage = re.search(pattern, tmp)
if lastPage:
    print('yes')
else:
    print('no')

no


In [59]:
mdf['link'][0]
# https://www.makro.co.za/appliances/fridges-freezers/commercial-fridges/commercial-fridges/fridgestar-772-l-sliding-double-door-fridge-/p/000000000000384903_EA
# https://www.makro.co.za/appliances/fridges-freezers/commercial-fridges/commercial-fridges/fridgestar-772-l-sliding-double-door-fridge-/p/000000000000384903_EA

'www.makro.co.za/appliances/fridges-freezers/commercial-fridges/commercial-fridges/fridgestar-772-l-sliding-double-door-fridge-/p/000000000000384903_EA'

In [16]:
print(f'{isPagination(True, 'https://www.makro.co.za/food/cooking/grains-rice-pasta/c/ICB?pageSize=20&q=%3Arelevance&page=0')}')

,price,name,delivery,store
30,R 1465,Royco Packet Marinade Chicken (1 x 40g),1,1
31,R 1465,Royco Packet Marinade Spare Rib (1 x 40g),1,1
32,R 1465,Royco Packet Marinade Smokey Bbq (1 x 40g),1,1
33,R 8175,Tastic Parboiled Rice (1 x 5kg),1,1
34,R 2495,Fatti's & Moni's Screws (1 x 500g),1,1
35,R 2495,Fatti's & Moni's Pasta Spirals (1 x 500g),1,1
36,R 5995,Iwisa Super Maize Meal (1 x 5kg),1,1
37,R 2115,Fatti's & Moni's Wholewheat Spaghetti High Fi...,1,1
38,R 39875,Fatti's & Moni's Wholewheat Macaroni High Fib...,1,1
39,R 2995,ACE Super Maize Meal (1 x 2.5kg),1,1
